Fallstudie:<br> 
**Erstellen eines Prognosemodells des Kreditkartenzahlungsverkehr für Online-Einkäufe**

In [1]:
#! mlflow server --host 127.0.0.1 --port 8080

In [2]:
import pandas as pd
pd.option_context('mode.use_inf_as_na', True)
pd.set_option('display.max_columns', 100)
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

import datetime as dt
import numpy as np
import re

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style='darkgrid',)
color_pal = sns.color_palette("muted")
sns.set_palette(color_pal)
sns.set_context("paper")
%matplotlib inline
# plot dimensions
plot_width = 12
plot_height = 8
palette_success ={0: color_pal[1], 1: color_pal[2]}

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score, precision_score, f1_score, recall_score

import mlflow
from mlflow.models import infer_signature

In [3]:
# parameter
rs = 23 # random state

# MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# date of today as string
dt_today = str(dt.date.today())

# Data preparation 2

In [4]:
df_trx = (pd.read_excel("../data/03_interim/df_trx_training.xlsx", sheet_name="df_trx")
          .drop(columns=["Unnamed: 0"])
          .rename(columns={"x_tmsp":"meta_tmsp"}))

In [5]:
# raw x- and y-variables
x_ = [x for x in list(df_trx.columns) if re.match("^x_", x)!=None]
y_ = [y for y in list(df_trx.columns) if re.match("^y_", y)!=None]
meta_ = [m for m in list(df_trx.columns) if m.find("meta_")!=-1]

In [6]:
df_trx[x_].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37612 entries, 0 to 37611
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   x_country                  37612 non-null  object 
 1   x_amount                   37612 non-null  int64  
 2   x_psp                      37612 non-null  object 
 3   x_3d_secured               37612 non-null  int64  
 4   x_card                     37612 non-null  object 
 5   x_tmsp_wd_str              37612 non-null  object 
 6   x_tmsp_daytime             37612 non-null  object 
 7   x_tmsp_h_sin               37612 non-null  float64
 8   x_tmsp_h_cos               37612 non-null  float64
 9   x_tmsp_wd_sin              37612 non-null  float64
 10  x_tmsp_wd_cos              37612 non-null  float64
 11  x_enc_country_Austria      37612 non-null  bool   
 12  x_enc_country_Germany      37612 non-null  bool   
 13  x_enc_country_Switzerland  37612 non-null  boo

Je nach Modell gibt es 2 Features-Sets:<br>

Für Modelle, die numerische Features benötigen: 
- 16 Features
- davon 6 numerische Features, die skaliert werden müssen
- und 10 Features vom Typ Boolean, die aus dem One Hot Encoding entstanden sind und als Integer encoded werden<br>

Für Modelle, bei denen der Typ der Features egal ist:
- 11 Features
- davon 6 numerische Features
- und 5 categorial Features vom Typ Object

In [7]:
# categorize features
feat_object_native = list(df_trx[x_].select_dtypes(include="object").columns) # original features of type object
#feat_object_bool_enc = list(df_trx[x_].select_dtypes(include="bool").columns) # encoded object features as bool
feat_num = list(df_trx[x_].select_dtypes(include="number").columns) # numerical features for scaling

## Train-Test-Split

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df_trx[x_], df_trx[y_], test_size=0.2, random_state=rs, shuffle=True)

## Feature Engineering Pipeline

In [9]:
# column transformer for models requiring numerical, scaled input features
coltransformer_4numerical = make_column_transformer(
    (MinMaxScaler(), make_column_selector(dtype_include="number")), 
    (OrdinalEncoder(), make_column_selector(dtype_include="bool"))
)

## Features

In [10]:
# for model with numerical input
x_train_numerical = coltransformer_4numerical.fit_transform(x_train)
x_test_numerical = coltransformer_4numerical.transform(x_test)

In [11]:
# for model with raw input
x_train_raw = x_train[feat_num + feat_object_native]
x_test_raw = x_test[feat_num + feat_object_native]

# Modeling
- Logistic regression: Scaling
- Naive Bayes
- KNN: Normalization
- Decision Tree
- SVM
- Random Forest

## Test design
Ziele:
- erfolgreiche Transaktionen in Abhängigkeit vom PSP (und den anderen Umständen) erkennen
- positiv = success

Vergleich der Modelle:
- Wir sind nicht nur an der Klassifikation an sich interessiert, sondern v.a. auch an der Wahrscheinlichkeit für einen Erfolg abhängig von der Wahrscheinlichkeit.
- AUC ist eine Vergleichsmetrik, die unabhängig vom gewählten Threshold ist.

Übergeordnete Metriken zur Bewertung der Performance:
- Precision meiner Vorhersage: keine Falschpositiven, um mehrfache Transaktionsgebühren zu sparen
- Recall/TPR/Sensitivity: möglichst viele erfolgreiche Transaktionen vorhersagen, um eine Auswahl an PSPs zu haben, um die Kosten minimieren zu können
- F1-Score als harmonisches Mittel

## Baseline model
2 Strategien kommen in Frage:
- most_frequent: macht wenig Sinn, weil dann kein Erfolg vorhergesagt wird, so dass Precision nicht berechnet werden kann. Recall/Sensitity ist ebenfalls = 0
- stratified

In [12]:
# Training and prediction
baseline_clf = DummyClassifier(strategy="stratified", random_state=rs)
baseline_clf.fit(x_train_raw, y_train)

y_pred_baseline = baseline_clf.predict(x_test_raw)
auc = roc_auc_score(y_test, y_pred_baseline)
prec = precision_score(y_test, y_pred_baseline, zero_division=np.nan)
recall = recall_score(y_test, y_pred_baseline, zero_division=np.nan)
f1 = f1_score(y_test, y_pred_baseline, zero_division=np.nan)

print(f"Baseline-model")
print(f"with AUC: {auc:.2f}")
print(f"with precision: {prec:.2f}")
print(f"with recall: {recall:.2f}")
print(f"and F1-score: {f1:.2f}")

Baseline-model
with AUC: 0.51
with precision: 0.21
with recall: 0.22
and F1-score: 0.21


In [14]:
# Create a new MLflow Experiment
mlflow.set_experiment(f"{dt_today}_Baseline")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    #mlflow.log_params(params)

    # Log the metric
    mlflow.log_metric("AUC", auc)
    mlflow.log_metric("Precision", prec)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1", f1)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Baseline model using stratified strategy")

    # Infer the model signature
    signature = infer_signature(x_train_raw, baseline_clf.predict(y_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=baseline_clf,
        artifact_path="baseline_model",
        signature=signature,
        input_example=x_train_raw,
        registered_model_name="baseline_model",
    )

2024/04/04 19:21:16 INFO mlflow.tracking.fluent: Experiment with name '2024-04-04_Baseline' does not exist. Creating a new experiment.
2024/04/04 19:21:16 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



KeyboardInterrupt: 